In [3]:
path = "/mnt/d/desktop/drone-mapping/data/drone-tiff.tif"

In [6]:
import rasterio as rio 
from rasterio.windows import Window
import os
import gc  # Garbage collection

In [ ]:
## Check it - Not working
def split_geotiff_with_overlap(input_file, output_dir="tiles", overlap_percent=10):
    """
    Split GeoTIFF into 4 tiles with optional overlap.
    
    Args:
        input_file (str): Path to the input GeoTIFF file
        output_dir (str): Directory to save the output tiles
        overlap_percent (int): Percentage of overlap between tiles
    """
    
    os.makedirs(output_dir, exist_ok=True)
    
    with rasterio.open(input_file) as src:
        height = src.height
        width = src.width
        
        # Calculate overlap in pixels
        overlap_h = int(height * overlap_percent / 200)  # Divide by 200 for half on each side
        overlap_w = int(width * overlap_percent / 200)
        
        # Calculate tile dimensions
        tile_height = height // 2
        tile_width = width // 2
        
        print(f"Original dimensions: {width} x {height}")
        print(f"Tile dimensions with overlap: {tile_width + 2*overlap_w} x {tile_height + 2*overlap_h}")
        
    tiles = [
        {
            "name": "top_left",
            "col_off": max(0, 0 - overlap_w),
            "row_off": max(0, 0 - overlap_h),
            "width": min(width, tile_width + overlap_w),
            "height": min(height, tile_height + overlap_h)
        },
        {
            "name": "top_right",
            "col_off": max(0, tile_width - overlap_w),
            "row_off": max(0, 0 - overlap_h),
            "width": min(width - tile_width + overlap_w, tile_width + overlap_w),
            "height": min(height, tile_height + overlap_h)
        },
        {
            "name": "bottom_left",
            "col_off": max(0, 0 - overlap_w),
            "row_off": max(0, tile_height - overlap_h),
            "width": min(width, tile_width + overlap_w),
            "height": min(height - tile_height + overlap_h, tile_height + overlap_h)
        },
        {
            "name": "bottom_right",
            "col_off": max(0, tile_width - overlap_w),
            "row_off": max(0, tile_height - overlap_h),
            "width": min(width - tile_width + overlap_w, tile_width + overlap_w),
            "height": min(height - tile_height + overlap_h, tile_height + overlap_h)
        }
    ]
    
    for i, tile in enumerate(tiles):
        window = Window(
            col_off=tile["col_off"],
            row_off=tile["row_off"],
            width=tile["width"],
            height=tile["height"]
        )
        
        data = src.read(window=window)
        transform = src.window_transform(window)
        
        base_name = os.path.splitext(os.path.basename(input_file))[0]
        output_file = os.path.join(output_dir, f"{base_name}_{tile['name']}_overlap.tif")
        
        with rasterio.open(
            output_file,
            'w',
            driver='GTiff',
            height=tile["height"],
            width=tile["width"],
            count=src.count,
            dtype=src.dtype,
            crs=src.crs,
            dtype = "int16",
            transform=transform,
            compress='lzw',
            tiled=True,
            blockxsize=512,
            blockysize=512
        ) as dst:
            dst.write(data)
        
        print(f"Created overlapping tile {i+1}/4: {output_file}")


SyntaxError: keyword argument repeated: dtype (2594217464.py, line 82)

## Splitting the image in 4 big tiles

Decreasing computational capacity and focusing on only one tile in order to test segmentation models

In [ ]:
### Working code with memory management 

## The code is splitting the image in 4 big tiles in order to decrease its size. 

input_cog = "/mnt/d/Desktop/drone-mapping/data/cog_3857.tif"
directory = "big-tiles"
os.makedirs(directory, exist_ok=True)
patch_size = 256
overlap = 4  # 2 pixels each side

def calculate_optimal_blocksize(width, height):
    """Calculate optimal block size based on tile dimensions"""
    # Use smaller blocks for large tiles to avoid memory issues
    if width > 2048 or height > 2048:
        return 256, 256
    elif width > 1024 or height > 1024:
        return 512, 512
    else:
        return 512, 512

def write_tile_safely(src, window, output_path, tile_width, tile_height):
    """Write a single tile with memory management"""
    try:
        # Read patch data
        patch_data = src.read(window=window)
        print(f"Patch shape: {patch_data.shape}")
        
        # Get transform
        transform = src.window_transform(window)
        
        # Calculate optimal block size
        blockxsize, blockysize = calculate_optimal_blocksize(tile_width, tile_height)
        
        # Write patch with optimized settings
        profile = src.profile.copy()
        profile.update({
            'height': tile_height,
            'width': tile_width,
            'transform': transform,
            'compress': 'JPEG',
            'tiled': True,
            'blockxsize': blockxsize,
            'blockysize': blockysize,
            'BIGTIFF': 'YES',  # Handle large files
        })
        
        with rio.open(output_path, 'w', **profile) as dst:
            dst.write(patch_data)
        
        # Explicitly delete patch data and force garbage collection
        del patch_data
        gc.collect()
        
        print(f"Successfully wrote: {output_path}")
        return True
        
    except Exception as e:
        print(f"Error writing tile {output_path}: {e}")
        return False

# Main processing with better memory management
with rio.open(input_cog) as src:
    print(f"Original image: {src.width}x{src.height}, {src.count} bands")
    print(f"Data type: {src.dtypes[0]}, CRS: {src.crs}")
    
    # Calculate tile dimensions (your original logic)
    tile_height = (src.height // 2) + overlap
    tile_width = src.width // 2 + overlap
    
    print(f"Tile dimensions: {tile_width}x{tile_height}")
    
    # Check if tiles are too large (warn user)
    tile_size_mb = (tile_width * tile_height * src.count * 8) / (1024 * 1024)  # Assuming 8 bytes per pixel
    print(f"Estimated tile size: {tile_size_mb:.1f} MB")
    
    if tile_size_mb > 500:  # More than 500MB per tile
        print("WARNING: Very large tiles detected. Consider reducing tile size.")
    
    tile_id = 1
    successful_tiles = 0
    
    for row in range(0, src.height, tile_height):
        for col in range(0, src.width, tile_width):
            print(f"\nProcessing tile {tile_id} - Row: {row}, Col: {col}")
            
            # Adjust window to stay within image bounds
            actual_width = min(tile_width, src.width - col)
            actual_height = min(tile_height, src.height - row)
            
            window = Window(
                col_off=col,
                row_off=row,
                width=actual_width,
                height=actual_height
            )
            
            print(f"Window: {window.width}x{window.height}")
            
            output_file_name = f"tile_{tile_id}.tif"
            output_path = os.path.join(directory, output_file_name)
            
            # Write tile safely
            if write_tile_safely(src, window, output_path, actual_width, actual_height):
                successful_tiles += 1
            
            tile_id += 1
            
            # Optional: Force garbage collection every few tiles
            if tile_id % 5 == 0:
                gc.collect()

print(f"\nCompleted: {successful_tiles} tiles written successfully")

Original image: 41451x68566, 3 bands
Data type: uint8, CRS: EPSG:3857
Tile dimensions: 20729x34287
Estimated tile size: 16267.4 MB

Processing tile 1 - Row: 0, Col: 0
Window: 20729x34287
Patch shape: (3, 34287, 20729)
Successfully wrote: big-tiles/tile_1.tif

Processing tile 2 - Row: 0, Col: 20729
Window: 20722x34287
Patch shape: (3, 34287, 20722)
Successfully wrote: big-tiles/tile_2.tif

Processing tile 3 - Row: 34287, Col: 0
Window: 20729x34279
Patch shape: (3, 34279, 20729)
Successfully wrote: big-tiles/tile_3.tif

Processing tile 4 - Row: 34287, Col: 20729
Window: 20722x34279
Patch shape: (3, 34279, 20722)
Successfully wrote: big-tiles/tile_4.tif

Completed: 4 tiles written successfully


## Patchify